In [102]:
import matplotlib.pyplot as plt
import numpy as np
from abc import ABC, abstractmethod
from scipy.stats import norm
from typing import Literal, Callable

In [103]:
INDEPENDENT_VARIABLES = ["strike_price", "underlying_price", "time", "interest_rate", "volatility"] # Independent variables for the Black-Scholes model
DEPENDENT_VARIABLES = ["option_price", "delta", "gamma", "theta", "vega", "rho", "vanna", "charm", "volga"] # Greeks

N = norm.cdf
phi = norm.pdf
at_the_forward_pricing = True  # Whether the option is at-the-forward (S = K * exp(-r * T))

def d1(S, K, T, r, sigma):
    return (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, r, sigma):
    return d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

def bs_call(S, K, T, r, sigma):
    return S * N(d1(S, K, T, r, sigma)) - K * np.exp(-r*T)* N(d2(S, K, T, r, sigma))

def bs_put(S, K, T, r, sigma):
    return K*np.exp(-r*T)*N(-d2(S, K, T, r, sigma)) - S*N(-d1(S, K, T, r, sigma))

def delta_call(S, K, T, r, sigma):
    return N(d1(S, K, T, r, sigma))

def delta_put(S, K, T, r, sigma):
    return N(d1(S, K, T, r, sigma)) - 1

def gamma(S, K, T, r, sigma):
    return phi(d1(S, K, T, r, sigma)) / (S * sigma * np.sqrt(T))

def theta_call(S, K, T, r, sigma):
    term1 = - (S * phi(d1(S, K, T, r, sigma)) * sigma) / (2 * np.sqrt(T))
    term2 = - r * K * np.exp(-r*T) * N(d2(S, K, T, r, sigma))
    return term1 + term2

def theta_put(S, K, T, r, sigma):
    term1 = - (S * phi(d1(S, K, T, r, sigma)) * sigma) / (2 * np.sqrt(T))
    term2 = r * K * np.exp(-r*T) * N(-d2(S, K, T, r, sigma))
    return term1 + term2

def vega(S, K, T, r, sigma):
    return S * phi(d1(S, K, T, r, sigma)) * np.sqrt(T)

def rho_call(S, K, T, r, sigma):
    return K * T * np.exp(-r*T) * N(d2(S, K, T, r, sigma))

def rho_put(S, K, T, r, sigma):
    return -K * T * np.exp(-r*T) * N(-d2(S, K, T, r, sigma))

def vanna(S, K, T, r, sigma):
    return phi(d1(S, K, T, r, sigma)) * d2(S, K, T, r, sigma) / sigma

def charm_call(S, K, T, r, sigma):
    return -phi(d1(S, K, T, r, sigma)) * (2 * r * T - d2(S, K, T, r, sigma) * sigma * np.sqrt(T)) / (2 * T * sigma * np.sqrt(T))

def charm_put(S, K, T, r, sigma):
    return -phi(d1(S, K, T, r, sigma)) * (2 * r * T + d2(S, K, T, r, sigma) * sigma * np.sqrt(T)) / (2 * T * sigma * np.sqrt(T))

def volga(S, K, T, r, sigma):
    return vega(S, K, T, r, sigma) * d1(S, K, T, r, sigma) * d2(S, K, T, r, sigma) / sigma

def greek_factory(dv, option_type: Literal["call", "put"]) -> Callable:
    if dv == "option_price":
        if option_type == "call":
            return bs_call
        elif option_type == "put":
            return bs_put
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "delta":
        if option_type == "call":
            return delta_call
        elif option_type == "put":
            return delta_put
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "gamma":
        return gamma
    elif dv == "theta":
        if option_type == "call":
            return theta_call
        elif option_type == "put":
            return theta_put
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "vega":
        if option_type == "call":
            return vega
        elif option_type == "put":
            return vega
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "rho":
        if option_type == "call":
            return rho_call
        elif option_type == "put":
            return rho_put
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "vanna":
        return vanna
    elif dv == "charm":
        if option_type == "call":
            return charm_call
        elif option_type == "put":
            return charm_put
        else:
            raise ValueError("option_type must be 'call' or 'put'")
    elif dv == "volga":
        return volga
    else:
        raise ValueError(f"Unsupported dependent variable: {dv}")

In [104]:

class Experiment(ABC):
    """
    Abstract base class for experiments.
    """
    IV: str
    DV: str
    equation: Callable
    x_values: np.ndarray
    y_values: np.ndarray

    option_type: Literal["call", "put"]

    at_the_forward = at_the_forward_pricing  # Whether the option is at-the-forward (S = K * exp(-r * T))

    K = [85, 100, 115]  # Strike prices for different experiments
    T = 1.0  # Time to expiration in years
    r = 0.05  # Interest rate
    sigma = 0.3  # Volatility
    S = 100 * np.exp(-r * T) if at_the_forward else 100  # Underlying asset price

    legend: np.ndarray

    def __init__(self, option_type: Literal["call", "put"],  dependent_variable: str, independent_variable: str):
        """
        Initialize the experiment with the given independent and dependent variables.
        """

        # Validate independent and dependent variables
        if independent_variable not in INDEPENDENT_VARIABLES:
            raise ValueError(f"Invalid independent variable: {independent_variable}. Must be one of {INDEPENDENT_VARIABLES}.")
        if dependent_variable not in DEPENDENT_VARIABLES:
            raise ValueError(f"Invalid dependent variable: {dependent_variable}. Must be one of {DEPENDENT_VARIABLES}.")
        
        self.IV = independent_variable
        self.DV = dependent_variable
        self.option_type = option_type
        self.equation = greek_factory(dependent_variable, option_type)
        self.legend = ["ITM", "ATM", "OTM"] if option_type == "call" else ["OTM", "ATM", "ITM"]
        if independent_variable == "strike_price":
            self.legend = ["Option Price"]

    @abstractmethod
    def solve(self):
        """
        Solve the experiment with the given independent and dependent variables. Store the results in x_values and y_values.
        """
        pass

    def plot_results(self):
        """
        Plot the results of the experiment.
        """
        plt.figure(figsize=(10, 6))
        for i in range(len(self.y_values)):
            plt.plot(self.x_values, self.y_values[i], label=self.legend[i])
        plt.xlabel(self.IV.capitalize())
        plt.ylabel(self.DV.capitalize())
        plt.title(f"{self.DV.capitalize()} as a function of {self.IV.capitalize()} for {str(self.option_type).capitalize()} Options")
        plt.legend()
        plt.grid()
        plt.show()

In [105]:
class StrikePriceExperiment(Experiment): # Vary strike price to see how it affects the Greeks
    def __init__(self, option_type: Literal["call", "put"], dependent_variable: str, independent_variable: str = "strike_price"):
        super().__init__(option_type, dependent_variable, independent_variable)
        self.x_values = np.linspace(50, 150, 100)  # Strike prices from 50 to 150
        self.y_values = []

    def solve(self):
        self.y_values.append([])
        for K in self.x_values:
            dependent_variable_sample = self.equation(self.S, K, self.T, self.r, self.sigma)
            self.y_values[-1].append(dependent_variable_sample)

class UnderlyingPriceExperiment(Experiment): # Vary underlying price to see how it affects the Greeks
    def __init__(self, option_type: Literal["call", "put"], dependent_variable: str, independent_variable: str = "underlying_price"):
        super().__init__(option_type, dependent_variable, independent_variable)
        self.x_values = np.linspace(50, 150, 100)  # Underlying prices from 50 to 150
        self.y_values = []

    def solve(self):
        for K in self.K:
            self.y_values.append([])
            for S in self.x_values:
                dependent_variable_sample = self.equation(S, K, self.T, self.r, self.sigma)
                self.y_values[-1].append(dependent_variable_sample)

class TimeExperiment(Experiment): # Vary time to see how it affects the Greeks
    def __init__(self, option_type: Literal["call", "put"], dependent_variable, independent_variable: str = "time"):
        super().__init__(option_type, dependent_variable, independent_variable)
        self.x_values = np.linspace(0.01, 1, 100)  # Time from 0.01 to 1 year
        self.y_values = []

    def solve(self):
        for K in self.K:
            self.y_values.append([])
            for T in self.x_values:
                dependent_variable_sample = self.equation(self.S, K, T, self.r, self.sigma)
                self.y_values[-1].append(dependent_variable_sample)

class InterestRateExperiment(Experiment): # Vary interest rate to see how it affects the Greeks
    def __init__(self, option_type: Literal["call", "put"], dependent_variable, independent_variable: str = "interest_rate"):
        super().__init__(option_type, dependent_variable, independent_variable)
        self.x_values = np.linspace(0.01, 0.1, 100)  # Interest rates from 0.01 to 0.1
        self.y_values = []

    def solve(self):
        for K in self.K:
            self.y_values.append([])
            for r in self.x_values:
                dependent_variable_sample = self.equation(self.S, K, self.T, r, self.sigma)
                self.y_values[-1].append(dependent_variable_sample)

class VolatilityExperiment(Experiment): # Vary volatility to see how it affects the Greeks
    def __init__(self, option_type: Literal["call", "put"], dependent_variable, independent_variable: str = "volatility"):
        super().__init__(option_type, dependent_variable, independent_variable)
        self.x_values = np.linspace(0.01, 1, 100)  # Volatility from 0.01 to 1
        self.y_values = []

    def solve(self):
        for K in self.K:
            self.y_values.append([])
            for sigma in self.x_values:
                dependent_variable_sample = self.equation(self.S, K, self.T, self.r, sigma)
                self.y_values[-1].append(dependent_variable_sample)


In [106]:
#############################################
#         INTERFACE FOR EXPERIMENTS         #
#############################################

import ipywidgets as widgets
from IPython.display import display, clear_output

# Static label to show the setting
forward_label = widgets.Label(
    value=f"at_the_forward_pricing: {at_the_forward_pricing}, current ATF value: 100"
)

# Dropdowns
option_type_widget = widgets.Dropdown(
    options=["call", "put"],
    value="call",
    description="Option Type:"
)

iv_widget = widgets.Dropdown(
    options=INDEPENDENT_VARIABLES,
    value="strike_price",
    description="Independent:"
)

dv_widget = widgets.Dropdown(
    options=DEPENDENT_VARIABLES,
    value="option_price",
    description="Dependent:"
)

generate_button = widgets.Button(
    description="Generate Plot",
    button_style='success'
)

output = widgets.Output()

# Function to run when button is clicked
def on_generate_click(b):
    with output:
        clear_output()
        
        option_type = option_type_widget.value
        iv = iv_widget.value
        dv = dv_widget.value
        
        # Choose correct experiment class
        experiment_class = {
            "strike_price": StrikePriceExperiment,
            "underlying_price": UnderlyingPriceExperiment,
            "time": TimeExperiment,
            "interest_rate": InterestRateExperiment,
            "volatility": VolatilityExperiment
        }[iv]
        
        try:
            experiment = experiment_class(option_type = option_type, dependent_variable=dv)
            experiment.solve()
            experiment.plot_results()
        except Exception as e:
            print(f"Error: {e}")

generate_button.on_click(on_generate_click)

# Display widgets
display(widgets.VBox([
    forward_label,
    option_type_widget,
    iv_widget,
    dv_widget,
    generate_button,
    output
]))
